In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from statsmodels.formula.api import ols
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
import matplotlib.pyplot as plt
import statsmodels.api as sm

Will do a pickle file of final df later

In [2]:
df= pd.read_csv('CraigslistCars.csv')

In [3]:
df_1 = df.copy()

In [4]:
df_2 = df_1.loc[(df_1['price']>100) & (df_1['price'] <59999)].copy()

In [5]:
df_2.drop('county', axis=1, inplace=True)

In [6]:
df_2.dropna(subset=['year'], axis =0,inplace=True)

In [7]:
df_condition_completed = df_2.loc[df_2['condition'].notna()].copy()
df_with_model_and_condition = df_condition_completed.loc[df_condition_completed['model'].notna()].copy()
condition_dict = {}
for a, b in df_with_model_and_condition[['model', 'condition']].values:
    condition_dict[a] = b

In [8]:
df_2['condition'] = df_2.apply(lambda x: x['condition'] if pd.isna(x['model']) or pd.notna(x['condition']) else condition_dict[x['model']] if x['model'] in condition_dict.keys() else x['condition'], axis=1)

In [9]:
df_cylinders_completed = df_2.loc[df_2['cylinders'].notna()].copy()
df_cylinders_and_model = df_cylinders_completed.loc[df_cylinders_completed['model'].notna()].copy()
cylinders_dictionary = {}
for c, d in df_cylinders_and_model[['model', 'cylinders']].values:
    cylinders_dictionary[c] = d

In [10]:
df_2['cylinders'] = df_2.apply( lambda x: x['cylinders'] if pd.isna( x['model'] ) or pd.notna( x['cylinders'] ) else cylinders_dictionary[x['model']] if x['model'] in cylinders_dictionary.keys() else x['cylinders'], axis=1)

In [11]:
df_2['cylinders'] = df_2.groupby('model')['cylinders'].ffill()

In [12]:
df_2['manufacturer'] = df_2.groupby('model')['manufacturer'].ffill()

In [13]:
df_2['fuel']=df_2.groupby('model')['fuel'].ffill()

In [14]:
df_2['odometer'] = df_2['odometer'].fillna(df_2.groupby(['year','model'])['odometer'].transform('mean'))

In [15]:
df_2.drop('VIN', axis=1, inplace=True)

In [16]:
df_2['size']= df_2.groupby('model')['size'].ffill()

In [17]:
df_2.drop('image_url', axis =1, inplace=True)

In [18]:
df_2.drop('posting_date', axis=1, inplace=True)

In [19]:
df_2.drop(['id', 'url','region_url'], axis=1, inplace=True)

In [20]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382885 entries, 27 to 426879
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        382885 non-null  object 
 1   price         382885 non-null  int64  
 2   year          382885 non-null  float64
 3   manufacturer  369287 non-null  object 
 4   model         378626 non-null  object 
 5   condition     362985 non-null  object 
 6   cylinders     348087 non-null  object 
 7   fuel          381410 non-null  object 
 8   odometer      382455 non-null  float64
 9   title_status  375626 non-null  object 
 10  transmission  381172 non-null  object 
 11  drive         265610 non-null  object 
 12  size          287063 non-null  object 
 13  type          300098 non-null  object 
 14  paint_color   269354 non-null  object 
 15  description   382883 non-null  object 
 16  state         382885 non-null  object 
 17  lat           379486 non-null  float64
 18  lon

In [21]:
df_2.drop(['lat', 'long'], axis=1, inplace=True)

In [22]:
df_2.drop(['paint_color', 'description', 'drive'], axis=1, inplace=True)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382885 entries, 27 to 426879
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        382885 non-null  object 
 1   price         382885 non-null  int64  
 2   year          382885 non-null  float64
 3   manufacturer  369287 non-null  object 
 4   model         378626 non-null  object 
 5   condition     362985 non-null  object 
 6   cylinders     348087 non-null  object 
 7   fuel          381410 non-null  object 
 8   odometer      382455 non-null  float64
 9   title_status  375626 non-null  object 
 10  transmission  381172 non-null  object 
 11  size          287063 non-null  object 
 12  type          300098 non-null  object 
 13  state         382885 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 43.8+ MB


In [23]:
df_3=df_2.copy()
df_3.dropna(inplace=True)
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206119 entries, 31 to 426875
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        206119 non-null  object 
 1   price         206119 non-null  int64  
 2   year          206119 non-null  float64
 3   manufacturer  206119 non-null  object 
 4   model         206119 non-null  object 
 5   condition     206119 non-null  object 
 6   cylinders     206119 non-null  object 
 7   fuel          206119 non-null  object 
 8   odometer      206119 non-null  float64
 9   title_status  206119 non-null  object 
 10  transmission  206119 non-null  object 
 11  size          206119 non-null  object 
 12  type          206119 non-null  object 
 13  state         206119 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 23.6+ MB


In [24]:
ohe=OneHotEncoder(categories='auto', sparse=False, drop='first')
ss=StandardScaler()

In [25]:
def onehotencoder(column_name, df):
    column_name_ohe = df[[column_name]]
    ohe = OneHotEncoder(categories = 'auto', sparse = False)
    ohe.fit(column_name_ohe)
    column_transformed = ohe.transform(column_name_ohe)
    column_encoded_ohe = pd.DataFrame(column_transformed, columns = ohe.get_feature_names([column_name]), index = df.index)
    return column_encoded_ohe

In [26]:
a =onehotencoder('region' , df_3)
b = onehotencoder('manufacturer' , df_3)
# c = onehotencoder('model' , df_3)
d = onehotencoder('condition' , df_3)
e = onehotencoder('cylinders' , df_3)
f = onehotencoder('fuel' , df_3)
g =onehotencoder('title_status' , df_3)
h = onehotencoder('transmission' , df_3)
i = onehotencoder('size' , df_3)
j = onehotencoder('type', df_3)
k = onehotencoder('state', df_3)

In [33]:
# X_train_ohe = X_train.select_dtypes('object')
# X_train_encoded = ohe.fit_transform(X_train_ohe)
# train_df = pd.DataFrame(X_train_encoded, columns=ohe.get_feature_names(), index=X_train.index)

In [27]:
df_4 = pd.concat([df_3,a,b,d,e,f,g,h,i,j,k], axis=1)

In [28]:
df_4.drop(['region', 'manufacturer','condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'size','type', 'state'], axis=1, inplace=True)

In [29]:
df_4.head()

,price,year,model,odometer,region_SF bay area,region_abilene,region_akron / canton,region_albany,region_albuquerque,region_altoona-johnstown,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
31,15000,2013.0,f-150 xlt,128000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,19900,2004.0,f250 super duty,88000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59,14000,2012.0,odyssey,95000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,22500,2001.0,f450,144700.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,15000,2017.0,charger rt 4dr sedan,90000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
X= df_4.drop(['model','price'], axis=1)
y= df_4['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
col_ss = ['year']
Scaled = ss.fit_transform(X_train[col_ss])
Scaled = pd.DataFrame(Scaled, index=X_train.index)
Scaled.columns = col_ss
X_train['year'] = Scaled['year']



<ipython-input-39-aa332f148755>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['year'] = Scaled['year']


In [40]:
col_sss = ['odometer']
Scaledd = ss.fit_transform(X_train[col_sss])
Scaledd = pd.DataFrame(Scaledd, index=X_train.index)
Scaledd.columns = col_sss
X_train['odometer'] = Scaledd['odometer']

<ipython-input-40-11956b13e498>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['odometer'] = Scaledd['odometer']


In [41]:
SimpleLinearRegression = LinearRegression()
SimpleLinearRegression.fit(X_train, y_train)
print(SimpleLinearRegression.score(X_train,y_train))

0.5009565902863771
